In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [2]:
import os
os.chdir('/content/drive/Shared drives/capstone_aa')
t=pd.read_csv('True.csv')
f=pd.read_csv('Fake.csv')

In [3]:
t['category'] = 1
f['category'] = 0

df = pd.concat([t,f])

In [4]:
from nltk.corpus import stopwords           #for using stopwords
from nltk.stem.porter import PorterStemmer  # for stemming
import nltk
nltk.download('stopwords')
import regex as re


from nltk.corpus import stopwords

cached_stopwords = stopwords.words('english')
porter = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
import regex as re

import nltk
from nltk.corpus import stopwords

cached_stopwords = stopwords.words('english')
porter = PorterStemmer()

In [6]:
def replace_spec(text):
    regex = r'[^a-zA-z0-9/s]'
    text = re.sub(regex, ' ', text)
    return text

def process_title(title):
  
    title = title.lower()
    title = replace_spec(title)
    title=re.sub("donald trump","trump ",title)    
  
    title_list = str.split(title)
    
    final_title =[]

    for item in title_list:
        if item not in cached_stopwords:
            final_title.append(item)
    
    return " ".join(final_title)

In [7]:
df['text_final'] = df.text.apply(process_title)

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)


In [10]:
df['t'] = df.text_final.apply(stemSentence)

In [11]:
x_train,x_test,y_train,y_test = train_test_split(df.t,df.category)
cv=CountVectorizer(min_df=0,max_df=1,ngram_range=(1,2))
#transformed train reviews
cv_train_reviews=cv.fit_transform(x_train)
#transformed test reviews
cv_test_reviews=cv.transform(x_test)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

BOW_cv_train: (33673, 1599009)
BOW_cv_test: (11225, 1599009)


In [12]:
model = Sequential()
model.add(Dense(units = 100 , activation = 'relu' , input_dim = cv_train_reviews.shape[1]))
model.add(Dense(units = 50 , activation = 'relu'))
model.add(Dense(units = 25 , activation = 'relu'))
model.add(Dense(units = 10 , activation = 'relu'))
model.add(Dense(units = 1 , activation = 'sigmoid'))

In [ ]:
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.fit(cv_train_reviews,y_train , epochs =5)

Epoch 1/5
33673/33673 [==============================] - 2787s 83ms/step - loss: 0.6914 - accuracy: 0.5232
Epoch 2/5
 6528/33673 [====>.........................] - ETA: 36:21 - loss: 0.1013 - accuracy: 0.9658

In [ ]:
pred = model.predict(cv_test_reviews)

In [ ]:
for i in range(len(pred)):
    if(pred[i] > 0.5):
        pred[i] = 1
    else:
        pred[i] = 0

In [ ]:
accuracy_score(pred,y_test)

In [ ]:
cm_cv = confusion_matrix(y_test,pred)
cm_cv